#### Reading the data from cosmosDB

In [0]:
%python
storage_account_name = "capstonecreditsa"
storage_account_key = "XHEwETA/1qyqzNTrS3ANoQSSOuOdyb1+5Cos2jOP/9rO+G9DDkKWME5UwIx82vO0uWegnJStyMXi+AStIXsSwQ=="
container_name = "transformed-data"
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)
input_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/"
 
df = spark.read.format("parquet").load(input_path)

#### Creating a temporary view of the data

In [0]:
%python
df.createOrReplaceTempView("data")

#### COUNTERS

In [0]:
%sql
select count(distinct(id)) as total_applicants from data;

total_applicants
36457


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select count(distinct(id)) as Males from data where gender='Male'

Males
12027


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select count(distinct(id)) as Females from data where gender='Female'

Females
24430


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    sum(CASE WHEN is_delay_month = 'Yes' THEN 1 END) AS Total_delayed_payments
FROM 
    data;


Total_delayed_payments
302229


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    sum(CASE WHEN is_delay_month = 'No' THEN 1 END) AS Total_non_delayed_payments
FROM 
    data;


Total_non_delayed_payments
475486


Databricks visualization. Run in Databricks to view.

#### 1)Total Applicants by Income Category

In [0]:
%sql
SELECT
        Income_category,
        COUNT(DISTINCT ID) AS unique_applicant_count
    FROM data
    GROUP BY Income_category


Income_category,unique_applicant_count
Student,11
Commercial associate,8490
State servant,2985
Working,18819
Pensioner,6152


Databricks visualization. Run in Databricks to view.

#### 2)Age Vs Avg_Credit_Card_tenure

In [0]:
%sql
WITH age_distribution AS (
    SELECT 
        ID,
        CASE 
            WHEN Age BETWEEN 18 AND 25 THEN '18-25'
            WHEN Age BETWEEN 26 AND 35 THEN '26-35'
            WHEN Age BETWEEN 36 AND 45 THEN '36-45'
            WHEN Age BETWEEN 46 AND 55 THEN '46-55'
            WHEN Age BETWEEN 56 AND 65 THEN '56-65'
            ELSE '66 and above'
        END AS age_group,
        MAX(MONTHS_AGO) - MIN(MONTHS_AGO) AS credit_card_tenure
    FROM 
        data
    GROUP BY 
        ID, Age
)
SELECT 
    age_group,
    AVG(credit_card_tenure) AS average_credit_card_tenure
FROM 
    age_distribution
GROUP BY 
    age_group
ORDER BY 
    age_group;


age_group,average_credit_card_tenure
18-25,13.219233903830482
26-35,19.565430085609457
36-45,21.247338087330274
46-55,21.25438919582566
56-65,20.157441016333937
66 and above,21.59198113207547


Databricks visualization. Run in Databricks to view.

#### 3)Gender-wise Delayed Payments

In [0]:
%sql
-- Doubt
    SELECT
        gender,
        AVG(CASE WHEN is_delay_month = 'Yes' THEN 1 ELSE 0 END) AS avg_delayed_count
    FROM data
    GROUP BY gender

gender,avg_delayed_count
Female,0.3895472881424532
Male,0.38673589220594595


Databricks visualization. Run in Databricks to view.

#### 4)% Delayed Payments Count per Customer

In [0]:
%sql
with my_cte as (
SELECT 
    ID,
    SUM(CASE WHEN is_delay_month = 'Yes' THEN 1 ELSE 0 END) AS delayed_payments,
    count(*) as total
FROM 
    data
GROUP BY 
    ID
order by delayed_payments
)
select id,delayed_payments*100/total as percent_delay_payments from my_cte



id,percent_delay_payments
5009865,0.0
5010515,0.0
5024501,0.0
5024845,0.0
5053715,0.0
5061461,0.0
5085835,0.0
5087733,0.0
5089298,0.0
5090761,0.0


Databricks visualization. Run in Databricks to view.

#### 5)Find the Outliers in the Annual_income

In [0]:
%sql

with my_cte as(
  select distinct(id),income_category,education_level,annual_income,
percentile_cont(0.25) within group(order by annual_income) over(partition by education_level) as Q1,
percentile_cont(0.75) within group(order by annual_income) over(partition by education_level) as Q3
from data
)
SELECT distinct(id),income_category,education_level,annual_income from my_cte
where annual_income< (Q1 - 1.5*(Q3-Q1)) or annual_income> (Q3 + 1.5*(Q3-Q1))
order by id,annual_income

id,income_category,education_level,annual_income
5009092,Working,Higher education,765000.0
5009094,Working,Higher education,765000.0
5009095,Working,Higher education,765000.0
5009133,Pensioner,Secondary,382500.0
5009134,Pensioner,Secondary,382500.0
5009135,Pensioner,Secondary,382500.0
5009136,Pensioner,Secondary,382500.0
5009137,Pensioner,Secondary,382500.0
5009138,Pensioner,Secondary,382500.0
5009139,Pensioner,Secondary,382500.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT distinct(id),income_category,education_level,annual_income 
from data

id,income_category,education_level,annual_income
5010147,Working,Secondary,157500.0
5021499,Working,Secondary,157500.0
5021995,Working,Higher education,135000.0
5026094,Working,Secondary,67500.0
5028325,Pensioner,Higher education,270000.0
5035537,Pensioner,Secondary,67500.0
5038573,Pensioner,Secondary,112500.0
5038649,Commercial associate,Secondary,135000.0
5041268,State servant,Higher education,99000.0
5041914,Working,Secondary,67500.0


Databricks visualization. Run in Databricks to view.

#### 6)percentage of delayed payments for each education level

In [0]:
%sql
WITH education_delay_data AS (
    SELECT 
        education_level,
        COUNT(CASE WHEN is_delay_month = 'Yes' THEN 1 END) AS delayed_payments,
        COUNT(*) AS total_payments
    FROM 
        data
    GROUP BY 
        education_level
)
SELECT 
    education_level,
    (delayed_payments * 100.0 / total_payments) AS percentage_delay_payments
FROM 
    education_delay_data
ORDER BY 
    percentage_delay_payments DESC;


education_level,percentage_delay_payments
Secondary,39.45153272892700
Incomplete higher,39.24956312440239
Higher education,37.63135845117561
Lower secondary,33.35644136337377
Academic degree,25.80645161290323


Databricks visualization. Run in Databricks to view.

#### 7)Age Distribution of Applicants with Percentage of Delayed Payments

In [0]:
%sql
-- The Applicants in the particular age has a percentage_total_delay_months of y...it means the percentage of total months for which the particular age group applicants setteled the loan after the due date
with my_cte as (
    SELECT 
        age,
        COUNT(CASE WHEN is_delay_month = 'Yes' THEN 1 END) AS delayed_payment_count,
        COUNT(*) AS total_count
from data
GROUP BY age
order by age
)
SELECT 
        age,round(delayed_payment_count*100.0/total_count,2) as percent_delay_count 
        
from my_cte


age,percent_delay_count
20,0.00
21,33.71
22,55.54
23,49.52
24,45.24
25,46.74
26,35.66
27,37.33
28,38.74
29,40.36


Databricks visualization. Run in Databricks to view.

#### 8)Most Common Job Types Among Delayed Payments

In [0]:
%sql
SELECT 
    job,
    COUNT(*) AS delayed_count
FROM 
    data
WHERE 
    is_delay_month = 'Yes'
GROUP BY 
    job
ORDER BY 
    delayed_count DESC;


job,delayed_count
Unknown,92914
Laborers,49614
Core staff,29880
Sales staff,27736
Managers,26844
Drivers,18213
High skill tech staff,12217
Medicine staff,10789
Accountants,10162
Cooking staff,5746


#### 9)Distribution of Credit Card Tenure Between the Applicants

In [0]:
%sql
WITH tenure_data AS (
    SELECT 
        (MAX(MONTHS_AGO) - MIN(MONTHS_AGO)) AS credit_card_tenure,
        id
    FROM 
        data
    GROUP BY 
        ID
)
SELECT 
    credit_card_tenure
FROM 
    tenure_data;


credit_card_tenure
49
21
1
19
42
19
6
21
7
27


Databricks visualization. Run in Databricks to view.

#### 10)Percentage of Delayed Payments by Employment Status

In [0]:
%sql
WITH delayed_payments AS (
    SELECT 
        employment_status,
        COUNT(CASE WHEN is_delay_month = 'Yes' THEN 1 END) AS delayed_payments,
        COUNT(*) AS total_payments
    FROM 
        data
    GROUP BY 
        employment_status
)
SELECT 
    employment_status,
    (delayed_payments * 100.0 / total_payments) AS percentage_delayed_payments
FROM 
    delayed_payments;


employment_status,percentage_delayed_payments
unemployed,38.98123324396783
employed,38.82523906961206


Databricks visualization. Run in Databricks to view.

#### 11)Average Credit Card Tenure by Employment Status

In [0]:
%sql
WITH tenure_data AS (
    SELECT 
        employment_status,
        (MAX(MONTHS_AGO) - MIN(MONTHS_AGO)) AS credit_card_tenure
    FROM 
        data
    GROUP BY 
        ID, employment_status
)
SELECT 
    employment_status,
    AVG(credit_card_tenure) AS avg_credit_card_tenure
FROM 
    tenure_data
GROUP BY 
    employment_status;


employment_status,avg_credit_card_tenure
unemployed,19.638616714697406
employed,20.54902454826866


Databricks visualization. Run in Databricks to view.

#### 12)Relationship Between Education Level and Credit Card Tenure

In [0]:
%sql
WITH tenure_by_education AS (
    SELECT 
       distinct(ID),
        education_level,
        (MAX(MONTHS_AGO) - MIN(MONTHS_AGO)) AS credit_card_tenure
    FROM 
        data
    GROUP BY 
        ID, education_level
    order by id
)

SELECT 
    education_level,
    AVG(credit_card_tenure) AS avg_credit_card_tenure
FROM 
    tenure_by_education
GROUP BY 
    education_level;



education_level,avg_credit_card_tenure
Secondary,20.159179884570367
Academic degree,25.15625
Incomplete higher,20.509929078014185
Lower secondary,22.141711229946523
Higher education,20.657846715328468


Databricks visualization. Run in Databricks to view.

#### 13)What is the gender-wise distribution of percentage of delayed payments across different income categories

In [0]:
%sql
WITH my_cte AS (
    SELECT 
        income_category,
        gender,
        COUNT(CASE WHEN is_delay_month = 'Yes' THEN 1 END) AS delayed_payment_count,
        COUNT(*) AS total_count
    FROM 
        data
    GROUP BY 
        income_category, gender
    ORDER BY income_category)
select income_category,gender,delayed_payment_count*100.0/total_count as percent_delay_count from my_cte;



income_category,gender,percent_delay_count
Commercial associate,Male,39.32875691122569
Commercial associate,Female,39.70184122288797
Pensioner,Female,38.81906677025730
Pensioner,Male,37.01479959972613
State servant,Female,39.27035041557031
State servant,Male,38.32260270070081
Student,Female,17.70186335403727
Student,Male,73.33333333333333
Working,Male,38.64161648398270
Working,Female,38.61485050012178


Databricks visualization. Run in Databricks to view.

#### 14)Trends in Credit Card Usage

In [0]:
%sql
SELECT 
    MONTHS_AGO,
    COUNT(DISTINCT ID) AS active_customers
FROM 
    data
GROUP BY 
    MONTHS_AGO
ORDER BY 
    MONTHS_AGO desc;

MONTHS_AGO,active_customers
60,321
59,627
58,955
57,1253
56,1588
55,1939
54,2279
53,2633
52,3070
51,3514


Databricks visualization. Run in Databricks to view.

#### 15)"How does the monthly credit card payment status distribution vary across different income categories?"

In [0]:
%sql
SELECT 
    income_category,
    STATUS,
    COUNT(*) AS month_count
FROM 
    data
GROUP BY 
    income_category, STATUS
ORDER BY 
    income_category, STATUS;

income_category,STATUS,month_count
Commercial associate,0,69302
Commercial associate,1,2368
Commercial associate,2,187
Commercial associate,3,77
Commercial associate,4,72
Commercial associate,5,561
Commercial associate,C,76295
Commercial associate,X,34523
Pensioner,0,47731
Pensioner,1,1157


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

#### 16)rank the top 5 applicants in each income_category based on the counts of delayed payments, non-delayed payments, and credit card tenure(More likely to get approved)


In [0]:
%sql
WITH payment_counts AS (
    SELECT ID,income_category,COUNT(CASE WHEN is_delay_month = 'Yes' THEN 1 END) AS delay_count,
        COUNT(CASE WHEN is_delay_month = 'No' THEN 1 END) AS no_delay_count,
        (MAX(MONTHS_AGO) - MIN(MONTHS_AGO)) AS credit_card_tenure
    FROM data 
    GROUP BY ID, income_category
),
ranked_applicants AS (
    SELECT 
        ID,
        income_category,
        delay_count,
        no_delay_count,
        credit_card_tenure,
        dense_rank() OVER (
            PARTITION BY income_category 
            ORDER BY delay_count ASC, no_delay_count DESC, credit_card_tenure DESC
        ) AS rank
    FROM 
        payment_counts
)
SELECT 
    ID,
    income_category,
    delay_count,
    no_delay_count,
    credit_card_tenure,
    rank
FROM 
    ranked_applicants
WHERE 
    rank <= 5
ORDER BY 
    income_category, rank;


ID,income_category,delay_count,no_delay_count,credit_card_tenure,rank
5118386,Commercial associate,0,61,60,1
5054311,Commercial associate,0,61,60,1
5089140,Commercial associate,0,61,60,1
5066716,Commercial associate,0,61,60,1
5041553,Commercial associate,0,60,59,2
5048395,Commercial associate,0,60,59,2
5010556,Commercial associate,0,59,58,3
5045994,Commercial associate,0,59,58,3
5142305,Commercial associate,0,59,58,3
5062195,Commercial associate,0,58,57,4


Databricks visualization. Run in Databricks to view.

#### 17)Identify the Top 3 Delayed Payment Applicants by Gender(More likely to get rejected)

In [0]:
%sql
WITH delayed_payments AS (
    SELECT 
        ID,
        gender,
        COUNT(CASE WHEN is_delay_month = 'Yes' THEN 1 END) AS delay_count
    FROM 
        data
    GROUP BY 
        ID, gender
),
ranked_delays AS (
    SELECT 
        ID,
        gender,
        delay_count,
        rank() OVER (
            PARTITION BY gender 
            ORDER BY delay_count desc
        ) AS rank
    FROM 
        delayed_payments
)
SELECT 
    ID,
    gender,
    delay_count,
    rank
FROM 
    ranked_delays
WHERE 
    rank <= 3
ORDER BY 
    gender, rank;


ID,gender,delay_count,rank
5048074,Female,61,1
5048073,Female,61,1
5053521,Female,60,3
5143259,Female,60,3
5145767,Female,60,3
5028256,Female,60,3
5024524,Female,60,3
5085886,Male,61,1
5021637,Male,59,2
5051087,Male,59,2


Databricks visualization. Run in Databricks to view.

#### 18)Determine the Top Applicants with the Longest Credit Card Tenure

In [0]:
%sql
WITH tenure_data AS (
    SELECT 
        ID,
        income_category,
        (MAX(MONTHS_AGO) - MIN(MONTHS_AGO)) AS credit_card_tenure
    FROM 
        data
    GROUP BY 
        ID, income_category
),
ranked_tenure AS (
    SELECT 
        ID,
        income_category,
        credit_card_tenure,
        ROW_NUMBER() OVER (
            PARTITION BY income_category 
            ORDER BY credit_card_tenure DESC
        ) AS rank
    FROM 
        tenure_data
)
SELECT 
    ID,
    income_category,
    credit_card_tenure,
    rank
FROM 
    ranked_tenure
WHERE 
    rank <= 5
ORDER BY 
    income_category, rank;


ID,income_category,credit_card_tenure,rank
5053221,Commercial associate,60,1
5041979,Commercial associate,60,2
5116236,Commercial associate,60,3
5061810,Commercial associate,60,4
5010021,Commercial associate,60,5
5095679,Pensioner,60,1
5010943,Pensioner,60,2
5112884,Pensioner,60,3
5041536,Pensioner,60,4
5105682,Pensioner,60,5


Databricks visualization. Run in Databricks to view.

#### 19)Find the Percentage of Each Status per Income Category

In [0]:
%sql
WITH status_counts AS (
    SELECT 
        income_category,
        STATUS,
        COUNT(DISTINCT ID) AS status_count
    FROM 
        data
    GROUP BY 
        income_category, STATUS
),
status_percentages AS (
    SELECT 
        income_category,
        STATUS,
        status_count,
        (status_count * 100.0 / SUM(status_count) OVER (PARTITION BY income_category)) AS percentage
    FROM 
        status_counts
)
SELECT 
    income_category,
    STATUS,
    status_count,
    percentage
FROM 
    status_percentages
ORDER BY 
    income_category, percentage DESC;


income_category,STATUS,status_count,percentage
Commercial associate,0,7380,42.16901891320496
Commercial associate,X,4603,26.30135420833095
Commercial associate,C,4162,23.78149820010285
Commercial associate,1,1057,6.03965487686418
Commercial associate,2,129,0.73710073710074
Commercial associate,3,65,0.37140734815153
Commercial associate,4,56,0.31998171533055
Commercial associate,5,49,0.27998400091423
Pensioner,0,5393,42.47460029928330
Pensioner,X,3372,26.55745451681500


Databricks visualization. Run in Databricks to view.

#### 20)Compare Average Delay Payments by Gender and Income Category(more likely to get rejected)

In [0]:
%sql
-- It means in each category who is more likely to get rejected...either male or female based on avg delay payments

WITH delay_counts AS (
    SELECT 
        ID,
        gender,
        income_category,
        COUNT(CASE WHEN is_delay_month = 'Yes' THEN 1 END) AS delay_count
    FROM 
        data
    GROUP BY 
        ID, gender, income_category
),
average_delays AS (
    SELECT 
        income_category,
        gender,
        AVG(delay_count) AS avg_delay_count,
        ROW_NUMBER() OVER (PARTITION BY income_category ORDER BY AVG(delay_count) DESC) AS rank
    FROM 
        delay_counts
    GROUP BY 
        income_category, gender
)
SELECT 
    income_category,
    gender,
    avg_delay_count,
    rank
FROM 
    average_delays
ORDER BY 
    income_category, rank;


income_category,gender,avg_delay_count,rank
Commercial associate,Male,8.563747040919852,1
Commercial associate,Female,8.538586661847098,2
Pensioner,Female,8.098779557589626,1
Pensioner,Male,7.740088105726873,2
State servant,Male,8.557251908396946,1
State servant,Female,8.551614370168258,2
Student,Male,11.0,1
Student,Female,5.7,2
Working,Male,8.274440677966101,1
Working,Female,8.173802866130723,2


Databricks visualization. Run in Databricks to view.